<a href="https://colab.research.google.com/github/meenakshi25jan/Prediction-Based-Music-Recommandation/blob/main/Predictionbasedrecommander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction Based  System using Apache Spark and Python


## Description

In this project we are going to  create a recommender system that will prediction base recommend new musical singers to a user. This reommandation system works based on their listening history. This system Suggest different songs or musical singers to a user is important to many music streaming services, such as Spotify ,Pandora and Netflix. 

for creating the system we are using Spark and the collaborative filtering technique. 
## Datasets

You will be using some publicly available song data from audioscrobbler, which can be found  given link(http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html). 
The original data file `user_singer_data.txt` contained about 141,000 unique users, and 1.6 million unique singers. About 24.2 million users’ plays of singers are recorded, along with their count.


The `singer_data.txt` file then provides a map from the canonical singer ID to the name of the singer.

## Package Imports (Require for Recommandtion)

In [ ]:
from pyspark.mllib.recommendation import *
import random
from operator import *

## Loading data

Load the three datasets into RDDs and name them `singerData`, `singerAlias`, and `usersingerAData`. 

In [ ]:
#Loading data into RDD
singerData = sc.textFile("singer_data_small.txt")
singerAlias = sc.textFile("singer_alias_small.txt")
usersingerData = sc.textFile("user_singer_data_small.txt")

alias_data = singerAlias.collect()
user_data = usersingerData.collect()
singer_canonical_dict = {}
user_list = []

for line in alias_data:
    singer_record = line.split("\t")
    singer_canonical_dict[singer_record[0]] = singer_record[1]

#Function to get canonical singer names
def canonicalsingerID(line):
    line = line.split(" ")
    
    if line[1] in singer_canonical_dict:
        return (int(line[0]),int(singer_canonical_dict[line[1]]),int(line[2]))
    else:
        return (int(line[0]),int(line[1]),int(line[2]))
    
#Getting canonical singer names        
usersingerData = usersingerData.map(canonicalsingerID)

#Creating allsingers dataset to be used later during model evaluation process
allsingers = usersingerData.map(lambda x:(x[1])).collect()
allsingers = list(set(allsingers))

## Data Exploration

In the blank below, write some code that with find the users' total play counts. Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an singer). Your output should look as follows:
```
User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.
```


In [ ]:
singer_data = singerAlias.collect()
    
user_play_count = {}
user_count_number = {}

for line in user_data:
     user_record = line.split()
     if user_record[0] in user_play_count:
         user_play_count[str(user_record[0])] = user_play_count[user_record[0]] + int(user_record[2])
         user_count_number[str(user_record[0])] = user_count_number[user_record[0]] + 1
     else:
         user_play_count[str(user_record[0])] = int(user_record[2])
         user_count_number[str(user_record[0])] = 1
top = 0
maximum = 2

for word, count in sorted(user_play_count.iteritems(), key=lambda (k,v): (v,k), reverse = True):
     if top > maximum:
        break
     print 'User ' + str(word) + ' has a total play count of ' + str(count) + ' and a mean play count of ' + str(count/user_count_number[word]) 
     top += 1

User 1059637 has a total play count of 674412 and a mean play count of 1878
User 2064012 has a total play count of 548427 and a mean play count of 9455
User 2069337 has a total play count of 393515 and a mean play count of 1519


####  Splitting Data for Testing

Use the [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function to divide the data (`usersingerData`) into:
* A training set, `trainData`, that will be used to train the model. This set should constitute 40% of the data.
* A validation set, `validationData`, used to perform parameter tuning. This set should constitute 40% of the data.
* A test set, `testData`, used for a final evaluation of the model. This set should constitute 20% of the data.

Use a random seed value of 13. Since these datasets will be repeatedly used you will probably want to persist them in memory using the [cache](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) function.

In addition, print out the first 3 elements of each set as well as their sizes; if you created these sets correctly, your output should look as follows:
```
[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031
```

In [ ]:
#Splitting the data into train, test and cross validation
trainData, validationData, testData = usersingerData.randomSplit([4, 4, 2], 13)

print trainData.take(3)
print validationData.take(3)
print testData.take(3)
print trainData.count()
print validationData.count()
print testData.count()

#Caching and creating ratings object
trainData = trainData.map(lambda l: Rating(*l)).cache()
validationData = validationData.map(lambda l: Rating(*l)).cache()
testData = testData.map(lambda l: Rating(*l)).cache()

[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031


## The Prediction based Recommender Model

For this project, we will train the model with implicit feedback. You can read more information about this from the collaborative filtering page: 

Therefore, we must first devise a way to evaluate models. Once we have a method for evaluation, we can run a parameter sweep, evaluate each combination of parameters on the validation data, and choose the optimal set of parameters. The parameters then can be used to make predictions on the test data.

### Model Evaluation

Although there may be several ways to evaluate a model, we will use a simple method here. Suppose we have a model and some dataset of *true* singer plays for a set of users. This model can be used to predict the top X singer recommendations for a user and these recommendations can be compared the singer that the user actually listened to (here, X will be the number of singer in the dataset of *true* singer plays). Then, the fraction of overlap between the top X predictions of the model and the X singer that the user actually listened to can be calculated. This process can be repeated for all users and an average value returned.

For example, suppose a model predicted [1,2,4,8] as the top X=4 singer for a user. Suppose, that user actually listened to the singer [1,3,7,8]. Then, for this user, the model would have a score of 2/4=0.5. To get the overall score, this would be performed for all users, with the average returned.

**NOTE: when using the model to predict the top-X singer for a user, do not include the singer listed with that user in the training data.**

Name your function `modelEval` and have it take a model (the output of ALS.trainImplicit) and a dataset as input. For parameter tuning, the dataset parameter should be set to the validation data (`validationData`). After parameter tuning, the model can be evaluated on the test data (`testData`).

In [ ]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from collections import defaultdict

#model evaluation function
def modelEval(model, dataset):
    global trainData
    global allsingers
    
    #Getting nonTrainsingers for each user
    usersingers = defaultdict(list)
    
    for data in trainData.collect():
        usersingers[data[0]].append(data[1])
        
    cvList = []
        
    for key in usersingers.keys():
        usersingers[key] = list(set(allsingers) - set(usersingers[key]))
        for singer in usersingers[key]:
            cvList.append((key, singer))
    
    #Creating user,nonTrainsingers RDD
    cvData = sc.parallelize(cvList)
    
    userOriginal = dataset.map(lambda x:(x.user, (x.product, x.rating))).groupByKey().collect()
    
    #prediction on the user, nonTrainsingers RDD
    predictions = model.predictAll(cvData)
    userPredictions = predictions.map(lambda x:(x.user, (x.product, x.rating))).groupByKey().collect()
    original = {}
    predictions = {}
    
    #Getting top X singers for each user
    for line in userOriginal:
        original[line[0]] = sorted(line[1], key=lambda x:x[1], reverse = True)
        
    for line in userPredictions:
        predictions[line[0]] = sorted(line[1], key=lambda x:x[1], reverse = True)
        
    similarity = []
    
    for key in userOriginal:
        similar = 0.0
        
        pred = predictions[key[0]]
        org = original[key[0]]
            
        for value in org:
            for item in pred[0:len(org)]:
                if (value[0] == item[0]):
                    similar += 1
                    break
                    
        #Similarity calculation        
        similarity.append(float(similar/len(org)))        
        
    string = "The model score for rank " + str(rank) + " is " + str(float(sum(similarity)/len(similarity)))    
    print string

### Model Construction

Now we can build the best model possibly using the validation set of data and the `modelEval` function. Although, there are a few parameters we could optimize, for the sake of time, we will just try a few different values for the [rank parameter] seed 345  Loop through the values [2, 10, 20] and figure out which one produces the highest scored based on your model evaluation function.

Important: this procedure may take several minutes to run.

For each rank value, print out the output of the `modelEval` function for that model. Your output should look as follows:
```
The model score for rank 2 is 0.090431
The model score for rank 10 is 0.095294
The model score for rank 20 is 0.090248
```

In [ ]:
#Model evaluation through different rank parameters
rank_list = [2, 10, 20]

for rank in rank_list:
    model = ALS.trainImplicit(trainData, rank, seed=345)
    modelEval(model,validationData)

The model score for rank 2 is 0.0909391661474
The model score for rank 10 is 0.0957125879247
The model score for rank 20 is 0.09047041725


Now, using the bestModel, we will check the results over the test data. Your result should be ~`0.0507`.

In [ ]:
bestModel = ALS.trainImplicit(trainData, rank=10, seed=345)
modelEval(bestModel, testData)

The model score for rank 20 is 0.0512321818394


## Trying Some singer Recommendations
Using the best model above, predict the top 5 singer for user `1059637` using the [recommendProducts](http://spark.apache.org/docs/1.5.2/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.recommendProducts) function. Map the results (integer IDs) into the real singer name using `singerAlias`. Print the results. The output should look as follows:
```
singer 0: Brand New
singer 1: Taking Back Sunday
singer 2: Evanescence
singer 3: Elliott Smith
singer 4: blink-182
```

In [ ]:
ratings = bestModel.recommendProducts(1059637, 5)

In [ ]:
import re
singer_data = singerData.collect()

singer_names_dict = {}

for line in singer_data:
    pattern = re.match( r'(\d+)(\s+)(.*)', line)
    singer_names_dict[str(pattern.group(1))] = pattern.group(3)

for i in range(0,5):
    if str(ratings[i].product) in singer_canonical_dict:
        singer_id = singer_canonical_dict[str(ratings[i].product)]
        print "singer " + str(i) + ": "  + str(singer_names_dict[str(singer_id)])
    else:
        print "singer " + str(i) + ": " + str(singer_names_dict[str(ratings[i].product)])

Artist 0: Brand New
Artist 1: Taking Back Sunday
Artist 2: Evanescence
Artist 3: Elliott Smith
Artist 4: blink-182
